<div align='center'>
<img src="https://raw.githubusercontent.com/EpistasisLab/tpot/master/images/tpot-logo.jpg" alt="drawing" width="300"/>
</div>
Simple TPOT Auto ML Classifier, No data preprocessing or Feature Engineering. Idea is to get a baseline !

Reference EDA : https://www.kaggle.com/subinium/tps-may-categorical-eda

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv',index_col=None)
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', index_col=None)
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', index_col=None)

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train

In [ ]:
test

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe()

In [ ]:
train['target'].value_counts()

Label Encode the target

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target_enc']=le.fit_transform(train['target'])

## Stratified split

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

ss_split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, val_index in ss_split.split(train, train['target_enc']):
    X_train = train.loc[train_index]
    X_val = train.loc[val_index]

In [ ]:
y_train = X_train['target_enc']
y_val = X_val['target_enc']
X_train.drop(['target', 'target_enc'],axis=1, inplace=True)
X_val.drop(['target', 'target_enc'],axis=1, inplace=True)

## TPOT AutoML

In [ ]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5, 
                      population_size=100, 
                      verbosity=2, 
                      scoring = 'roc_auc_ovr',
                      random_state=42,
                      config_dict='TPOT cuML')
tpot.fit(X_train, y_train)
print('Validation score :', tpot.score(X_val, y_val))
tpot.export('tps-may-2021.py')

In [ ]:
preds = tpot.predict_proba(test)

In [ ]:
sample = submission

In [ ]:
sample = pd.DataFrame(np.column_stack((sample['id'],preds)), index=sample.index, columns=sample.columns)

In [ ]:
sample.dtypes

In [ ]:
sample['id'] = sample['id'].astype(int)

In [ ]:
sample.to_csv('tps_may_tpot_baseline.csv',index=None)